In [81]:
import advent
data: list[str] = advent.get_lines(20)

def parse_rule(rule: str):
    l, r = rule.split(' -> ')
    if l == 'broadcaster':
        ntype, nname = 'b', 'broadcaster'
    else:
        ntype, nname = l[0], l[1:]
    destinations = r.split(', ')
    return nname, ntype, destinations

rules = [parse_rule(rule) for rule in data]
node_dict = dict((a[0], (a[1], a[2])) for a in rules)
#rules, node_dict


In [82]:
from collections import deque

def connected_inputs(node_dict: dict[str, tuple[str, list[str]]]):
    # dictionary from nodes to connected input nodes
    result: dict[str, list[str]] = {}
    for key in node_dict:
        for dest in node_dict[key][1]:
            if dest not in result:
                result[dest] = [key]
            else:
                result[dest].append(key)
    return result

# This will be read-only so can be global variable
inputs = connected_inputs(node_dict)

def reset_states():
    # This function creates a new state. Basically, starting fresh from 0 button presses
    #messages = deque([('broadcaster', 0, 'input')])
    states: dict[str, int|dict[str, int]] = dict((node, 0) for node in node_dict)
    for node in node_dict:
        if node_dict[node][0] == '&':
            states[node] = dict((s, 0) for s in inputs[node])
    return states

In [87]:
low, high = 0, 0
result_loops: dict[str, int] = {}

def handle_message(
        messages: deque[tuple[str, int, str]],
        states: dict[str, int|dict[str, int]],
        debug: tuple[int, str] = (0, 'never')):
    # INPLACE updates the message queue by taking a message and processing it
    # also INPLACE updates the states dictionary
    global low
    global high
    global result_loops

    message = messages.popleft()
    node, content, sender = message

    # Store the number of signals sent. only for part 1
    if content == 0: low += 1
    elif content == 1: high += 1

    if node == debug[1] and content == 1:
        # Store the result of a found loop. only for part 2
        if sender not in result_loops: result_loops[sender] = debug[0]
    if node == 'rx' and content == 0 and debug[0] > 0:
        # Tried to brute force part 2, didn't work, this never prints anything :P
        print(f"Node rx received 0 from {sender} at step {debug[0]}!")

    if node not in node_dict:
        return messages, states # this is an output-only node

    node_type, destinations = node_dict[node]

    if node_type == 'b':
        for d in destinations:
            messages.append((d, content, 'broadcaster'))
    elif node_type == '%' and content == 0:
        states[node] = 1 - states[node]
        for d in destinations:
            messages.append((d, states[node], node))
    elif node_type == '&':
        states[node][sender] = content
        m = 0 if all(states[node].values()) else 1
        for d in destinations:
            messages.append((d, m, node))
    return messages, states

In [84]:
states = reset_states()

low, high = 0, 0

for _ in range(1000):
    messages = deque([('broadcaster', 0, 'input')])
    while messages:
        handle_message(messages, states)

low * high

825896364

In [85]:
# This could only really be found by inspecting the input
# Basically, the node that sends to rx is an '&' node with multiple inputs
# so we want all those inputs to be '1'. However, this happens rarely.
# we debug them to find when they are sending '1's
to_debug = inputs['rx'][0]

# Reset messages/states since it was contaminated from part 1
states = reset_states()

result_loops = {}

buttons = 0
for _ in range(100_000):
    messages = deque([('broadcaster', 0, 'input')])
    buttons += 1
    while messages:
        handle_message(messages, states, debug=(buttons, to_debug))

In [86]:
import math
# Okay dont ask too many questions, but mainly, I took the input, removed a bunch of lines (not random, handpicked)
# and got advent20.simplified.txt
# Then I ran that and it got a low on rx at like 50.000 steps. So I found the loops in the input there,
# and found you could just take the lcm to calculate the place where all the loops intersect
math.lcm(*result_loops.values())

243566897206981